# Data Wrangling: Clean, Transform, Merge, Reshape

In [1]:
import numpy as np
import pandas as pd
from pandas import *

## 으로 시작하는 주석이 있는 코드 블럭은 시간이 허락하면 하도록 하겠습니다.
### 은 자습해도 될 것 같습니다.

## Combining and Merging Data Sets
다음과 같은 내용을 다룹니다.
* pandas.merge (aka _join_)
* pandas.concat
* combine_first

### Database-style DataFrame Merges
#### 다 대 1 머지

In [2]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [3]:
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [4]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [5]:
# 키를 명시적으로 지정해 줍니다.
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [6]:
# 좌/우 각각 따로 키를 지정해 줍니다.
df1_left = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2_right = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})
pd.merge(df1_left, df2_right, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


__`merge`__는 default로 inner join을 수행합니다.
그러나 join에는 outer join이란 것도 있고, outer join에는 다시 left outer join, right outer join 및 full outer join이 있습니다.

그 외 여러가지 join에 관한 자세한 사항은 [Join (SQL)](https://ko.wikipedia.org/wiki/Join_(SQL%29)을 참고하시기 바랍니다. __`merge`__에서는 다음과 같이 join 방식을 지정할 수 있으며, 그 종류는 'inner', 'left', 'right', 'outer'가 있습니다.

In [7]:
# outer 머지
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,3,c,NaN
7,NaN,d,2


#### 다 대 다 머지

In [8]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [9]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [10]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,2,a,0
5,2,a,2
6,3,c,NaN
7,4,a,0
8,4,a,2
9,5,b,1


In [11]:
# 머지 키로 여러 컬럼을 사용할 수도 있습니다.
left = DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'], 'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 'key2': ['one', 'one', 'one', 'two'], 'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,NaN
3,bar,one,3,6
4,bar,two,NaN,7


In [12]:
## 키가 아닌 컬럼명이 아래와 같이 중복될 경우,
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [13]:
## 중복된 컬럼명 뒤에 접미사를 각각 지정해 줘서 구분이 용이하도록 할 수 있습니다.
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index
컬럼명이 아닌 인덱스를 이용하여 머지를 수행할 경우입니다.

In [14]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [15]:
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [16]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [17]:
## outer join을 수행합니다.
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [18]:
## 인덱스에 계층이 있는 경우
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 'key2': [2000, 2001, 2002, 2001, 2002], 'data': np.arange(5.)})
lefth

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


In [19]:
## 인덱스에 계층이 있는 경우 (continued)
righth = DataFrame(np.arange(12).reshape((6, 2)), index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'], [2001, 2000, 2000, 2000, 2001, 2002]], columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [20]:
## 인덱스에 계층이 있는 경우 (continued)
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1


In [21]:
## index에 계층이 있는 경우 (continued)
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1
4,4,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2,3


In [22]:
# 양쪽 인덱스 이용하여 머지를 수행할 경우
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'], columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [23]:
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]], index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [24]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


In [25]:
# DataFrame 객체의 join 메소드를 이용해 머지를 수행할 수 도 있습니다.
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


`pd.merge(left, right)`는 `left.join(right, how='inner')`와 같습니다. __`merge`__ 함수의 기본 join 방법은 inner join이기 때문입니다.

`left.join(right)`는 `pd.merge(left, right, how='left')`와 같습니다. __`join`__ 메소드의 기본 join 방법은 left outer join이기 때문입니다.

실제로 __`join`__ 메소드의 소스를 까보면 그 안에서 __`pd.merge`__ 함수를 사용합니다.

In [26]:
# 이전 버전과의 호환성 문제로 DataFrame 객체의 join 메소드는 기본적으로 left join을 수행합니다.
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [27]:
## join 메소드의 파라미터로 DataFrame의 리스트를 넘길 수 도 있습니다.
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]], index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,NaN,NaN,7,8
c,3,4,9,10,9,10
e,5,6,13,14,11,12


### Concatenating Along an Axis

In [28]:
# NumPy의 concatenate 함수를 봅시다.
arr = np.arange(12).reshape((3, 4))
print arr
np.concatenate([arr, arr], axis=1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

pandas의 객체인 __`Series`__와 __`DataFrame`__을 놓고 보면 concatenation을 수행할 때 다음과 같은 의문이 듭니다.
* 객체의 index가 서로 다르게 부여되었을 경우 이들 간의 합집합(union)이나 교집합(intersection) 연산이 가능한가?
* 결과 객체에서 그룹을 식별할 필요가 있는가? (잉?)
* 과연 concatenation에 축(axis)만이 문제인가?

In [29]:
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [30]:
# 기본적으로 concat 함수는 axis=0이 default 파라미터 입니다. axis=1을 주면 결과는 Series가 아니라 DataFrame이 됩니다.
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [31]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [32]:
# axis=1로 concatenation을 수행하는 것은 바로 outer join입니다.
pd.concat([s1, s4], axis=1)

,0,1
a,0,0
b,1,5
f,NaN,5
g,NaN,6


In [33]:
# inner join도 가능합니다.
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [34]:
# 축으로 사용할 인덱스를 나열해 줄 수도 있습니다.
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0,0
c,NaN,NaN
b,1,5
e,NaN,NaN


In [35]:
# axix=1일 때 keys 파라미터를 통해 컬럼명을 지정해 줄 수 있습니다.
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [36]:
# 동일한 로직을 DataFrame에도 적용할 수 있습니다.
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1      5    6
b      2   3    NaN  NaN
c      4   5      7    8

In [37]:
# 데이터프레임 객체의 리스트와 keys를 사용하지 않고, 사전 객체를 이용할 수도 있습니다.
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1      5    6
b      2   3    NaN  NaN
c      4   5      7    8

In [38]:
# names 파라미터를 통해 계층화된 인덱스에 이름을 부여할 수 있습니다.
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1      5    6
b          2   3    NaN  NaN
c          4   5      7    8

In [39]:
# 데이터프레임 객체의 row 인덱스가 의미 없을 경우 ignore_index 파라미터를 사용합니다.
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

,a,b,c,d
0,0.351509,-1.179583,1.674942,-0.144818
1,1.651936,-0.743200,0.069257,0.125162
2,0.181548,-1.191632,0.851014,0.007547


In [40]:
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,0.322562,-1.088520,-0.037751
1,-0.033167,0.658064,1.013566


In [41]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.351509,-1.179583,1.674942,-0.144818
1,1.651936,-0.743200,0.069257,0.125162
2,0.181548,-1.191632,0.851014,0.007547
3,-0.037751,0.322562,NaN,-1.088520
4,1.013566,-0.033167,NaN,0.658064


### Combining Data with Overlap

In [42]:
# NumPy의 where 함수를 이용해 봅시다.
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
print '==== a ===='
print a
b = Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print '==== b ===='
print b

==== a ====
f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
==== b ====
f     0
e     1
d     2
c     3
b     4
a   NaN
dtype: float64


In [43]:
# np.where(condition, then, else)
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [44]:
# Series 객체의 conbine_first 메소드를 사용해 봅시다.
print '==== b[:-2] ===='
print b[:-2]
print '==== a[2:] ===='
print a[2:]
b[:-2].combine_first(a[2:])

==== b[:-2] ====
f    0
e    1
d    2
c    3
dtype: float64
==== a[2:] ====
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64


a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [45]:
# DataFrame 객체에도 conbine_first 메소드를 사용해 봅시다.
df1 = DataFrame({'a': [1., np.nan, 5., np.nan], 'b': [np.nan, 2., np.nan, 6.], 'c': range(2, 18, 4)})
df1

,a,b,c
0,1,NaN,2
1,NaN,2,6
2,5,NaN,10
3,NaN,6,14


In [46]:
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.], 'b': [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5,NaN
1,4,3
2,NaN,4
3,3,6
4,7,8


In [47]:
df1.combine_first(df2)

,a,b,c
0,1,NaN,2
1,4,2,6
2,5,4,10
3,3,6,14
4,7,8,NaN


## Reshaping and Pivoting
### Reshaping with Hierarchical Indexing
다음 두가지 액션에 대해 알아봅시다.
* __`stack`__: columns into rows
* __`unstack`__: rows into columns

In [48]:
data = DataFrame(np.arange(6).reshape((2, 3)), index=pd.Index(['Ohio', 'Colorado'], name='state'), columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [49]:
# stack에 의해 one, two, three 컬럼이 각각 행으로 됩니다.
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [50]:
# unstack에 의해 원상복구 됩니다.
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [51]:
## unstack을 할 때 컬럼을 index나 컬럼명을 지정해 줄 수도 있습니다.
## 숫자로 지정할 때 0은 인덱스를 의미합니다.
## 아래 두 결과는 같습니다.
result.unstack(0)
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [52]:
## 행의 모든 컬럼에 값이 없으면 (NaN이면, 즉 missing이면) 그 행은 제거됩니다.
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
print data2
data2.unstack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64


,a,b,c,d,e
one,0,1,2,3,NaN
two,NaN,NaN,4,5,6


In [53]:
## stack을 수행할 때 dropna 파라미터를 False로 하면 missing 데이터를 제거하지 않을 수도 있습니다.
print data2.unstack().stack()
print data2.unstack().stack(dropna=False)

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: float64
one  a     0
     b     1
     c     2
     d     3
     e   NaN
two  a   NaN
     b   NaN
     c     4
     d     5
     e     6
dtype: float64


In [54]:
# DataFrame을 unstack해 봅시다.
df = DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [55]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [56]:
# 다시 side 컬럼으로 stack해 봅시다.
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

### Pivoting "long" to "wide" Format

In [57]:
ldata_str = """1959-03-31,realgdp,2710.349
1959-03-31,infl,0.000
1959-03-31,unemp,5.800
1959-06-30,realgdp,2778.801
1959-06-30,infl,2.340
1959-06-30,unemp,5.100
1959-09-30,realgdp,2775.488
1959-09-30,infl,2.740
1959-09-30,unemp,5.300
1959-12-31,realgdp,2785.204
1959-12-31,infl,0.270
1959-12-31,unemp,5.600"""

ldata_date = [line.split(',')[0] for line in ldata_str.split('\n')]
ldata_item = [line.split(',')[1] for line in ldata_str.split('\n')]
ldata_value = [line.split(',')[2] for line in ldata_str.split('\n')]

# 같은 날짜에 item 항목이 다르게 value가 부여되어 있습니다.
ldata = pd.DataFrame({'date': ldata_date, 'item': ldata_item, 'value': ldata_value})
ldata

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [58]:
# 실제로 item은 컬럼이므로 피봇해보겠습니다.
# 첫번째 인자는 하나로 합칠 컬럼, 두번째 인자는 항목 이름, 세번째 인자는 항목의 값
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31,0.000,2710.349,5.800
1959-06-30,2.340,2778.801,5.100
1959-09-30,2.740,2775.488,5.300
1959-12-31,0.270,2785.204,5.600


In [59]:
ldata['value2'] = np.random.randn(len(ldata))
ldata

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,0.848012
1,1959-03-31,infl,0.000,0.569919
2,1959-03-31,unemp,5.800,-0.580687
3,1959-06-30,realgdp,2778.801,0.839717
4,1959-06-30,infl,2.340,0.020976
5,1959-06-30,unemp,5.100,-1.745428
6,1959-09-30,realgdp,2775.488,0.185849
7,1959-09-30,infl,2.740,-0.425815
8,1959-09-30,unemp,5.300,-0.043004
9,1959-12-31,realgdp,2785.204,0.032678


In [60]:
## 값이 여러가지 일 때 어떻게 피봇되는지 봅시다.
pivoted = ldata.pivot('date', 'item')
pivoted

value                     value2                    
item         infl   realgdp  unemp      infl   realgdp     unemp
date                                                            
1959-03-31  0.000  2710.349  5.800  0.569919  0.848012 -0.580687
1959-06-30  2.340  2778.801  5.100  0.020976  0.839717 -1.745428
1959-09-30  2.740  2775.488  5.300 -0.425815  0.185849 -0.043004
1959-12-31  0.270  2785.204  5.600 -0.370491  0.032678  1.744459

In [61]:
## 이 중 원하는 값만 가져와서 따로 DataFrame을 만듭니다.
pivoted['value']

item,infl,realgdp,unemp
date,,,
1959-03-31,0.000,2710.349,5.800
1959-06-30,2.340,2778.801,5.100
1959-09-30,2.740,2775.488,5.300
1959-12-31,0.270,2785.204,5.600


In [62]:
### pivot은 set_index로 계층적 인덱스를 만들고 unstak한 것입니다.
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked

value                     value2                    
item         infl   realgdp  unemp      infl   realgdp     unemp
date                                                            
1959-03-31  0.000  2710.349  5.800  0.569919  0.848012 -0.580687
1959-06-30  2.340  2778.801  5.100  0.020976  0.839717 -1.745428
1959-09-30  2.740  2775.488  5.300 -0.425815  0.185849 -0.043004
1959-12-31  0.270  2785.204  5.600 -0.370491  0.032678  1.744459

## Data Transformation
### Removing Duplicates

In [63]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4, 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [64]:
# 중복 여부를 인덱싱하거나 중복을 제거합니다.
print data.duplicated()
data.drop_duplicates()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool


,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [65]:
## 중복 제거에는 모든 컬럼을 사용하는 것이 default입니다만, 당연히 중복을 판단할 컬럼을 지정할 수도 있습니다.
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [66]:
## 중복 제거 시 default는 먼저 나타난 것을 남기고 뒤에 중복된 것을 제거합니다만, 맨 마지막 것을 남길 수도 있습니다.
data.drop_duplicates(['k1', 'k2'], take_last=True)

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [67]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'], 'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [68]:
# map은 Series의 모든 원소에 함수를 적용하여 새로운 Series를 만들어 냅니다.
meat_to_animal = {'bacon': 'pig', 'pulled pork': 'pig', 'pastrami': 'cow', 'corned beef': 'cow', 'honey ham': 'pig', 'nova lox': 'salmon'}
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [69]:
## 당연히 lambda 함수를 사용할 수 있습니다.
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

In [70]:
# replace는 Series의 모든 원소에 해당 값(좌측 파라미터)을 다른 값(우측 파라미터)으로 바꿔서 새로운 Series를 만들어 냅니다.
data = Series([1., -999., 2., -999., -1000., 3.])
print data
data.replace(-999, np.nan)

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: float64


0       1
1     NaN
2       2
3     NaN
4   -1000
5       3
dtype: float64

In [71]:
## 여러 값을 바꿀 때에는 길이가 같은 두개의 리스트를 넘기거나, 사전을 넘기면 됩니다.
## 아래 두 결과는 같습니다.
data.replace([-999, -1000], [np.nan, 0])
data.replace({-999: np.nan, -1000: 0})

0     1
1   NaN
2     2
3   NaN
4     0
5     3
dtype: float64

### Renaming Axis Indexes

In [72]:
data = DataFrame(np.arange(12).reshape((3, 4)), index=['Ohio', 'Colorado', 'New York'], columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [73]:
# 인덱스에도 map을 할 수 있습니다.
data.index.map(str.upper)

array(['OHIO', 'COLORADO', 'NEW YORK'], dtype=object)

In [74]:
# 아래와 같이 인덱스를 바꿀 수도 있습니다.
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [75]:
# rename 메소드를 사용하면 원본을 수정하지 않고 새로운 데이터프레임 객체를 만들 수 있습니다.
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [76]:
## rename 메소드에 함수 대신 사전을 사용할 수도 있습니다.
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [77]:
### inplace 파라미터를 True로 하면 자기 자신을 수정할 수도 있습니다.
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### Discretization and Binning

In [78]:
# 연속적인 값을 구간별로 나눠서 binning을 수행합니다.
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
print cats
print '==== codes and categories ===='
print cats.codes    # .labels는 deprecated 되었습니다.
print cats.categories    # .levels는 deprecated 되었습니다.

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]
==== codes and categories ====
[0 0 0 1 0 0 2 1 3 2 2 1]
Index([u'(18, 25]', u'(25, 35]', u'(35, 60]', u'(60, 100]'], dtype='object')


In [79]:
# 위 경우는 왼쪽이 open(exclusive)이며 오른쪽이 closed(inclusive)입니다.
# 기본은 오른쪽 inclusive 여부가 True입니다만, False로 줄 수 있습니다.
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, object): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [80]:
# 각 그룹에 이름을 부여할 수 있습니다.
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [81]:
## 최소값과 최대값을 이용해서 자동으로 4개의 구간으로 나눕니다.
## precision 파라미터는 정확도로 소수점 이하 자리수입니다.
data = np.random.rand(10)
pd.cut(data, 4, precision=2)

[(0.18, 0.38], (0.78, 0.98], (0.58, 0.78], (0.78, 0.98], (0.78, 0.98], (0.58, 0.78], (0.58, 0.78], (0.58, 0.78], (0.78, 0.98], (0.78, 0.98]]
Categories (4, object): [(0.18, 0.38] < (0.38, 0.58] < (0.58, 0.78] < (0.78, 0.98]]

In [82]:
## cut이 단순히 값을 4개 구간으로 나눈 데 반해, qcut은 각 구간의 개체수가 동일하도록 나눠줍니다.
data = np.random.rand(1000)
pd.qcut(data, 4)

[[0.000561, 0.231], (0.231, 0.491], (0.231, 0.491], (0.491, 0.738], [0.000561, 0.231], ..., (0.738, 0.999], (0.231, 0.491], (0.231, 0.491], (0.738, 0.999], (0.231, 0.491]]
Length: 1000
Categories (4, object): [[0.000561, 0.231] < (0.231, 0.491] < (0.491, 0.738] < (0.738, 0.999]]

In [83]:
## quantile 구간을 갯수가 아니라 직접 지정해 줄 수도 있습니다. 0부터 1까지 적산한 구간입니다.
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[[0.000561, 0.0866], (0.0866, 0.491], (0.0866, 0.491], (0.491, 0.905], (0.0866, 0.491], ..., (0.905, 0.999], (0.0866, 0.491], (0.0866, 0.491], (0.491, 0.905], (0.0866, 0.491]]
Length: 1000
Categories (4, object): [[0.000561, 0.0866] < (0.0866, 0.491] < (0.491, 0.905] < (0.905, 0.999]]

### Detecting and Filtering Outliers

In [84]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [85]:
# 컬럼 인덱스 3번 중에서 절대값이 3보다 큰 값을 찾아보겠습니다.
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [86]:
# 절대값이 3보다 큰 값을 갖는 컬럼이 존재하는 모든 행을 찾아보겠습니다.
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [87]:
# 절대값이 3보다 큰 값은 절대값이 3이 되도록 치환해보겠습니다.
data[np.abs(data) > 3] = np.sign(data) * 3    # np.sign() 함수는 값의 양, 음 여부에 따라 각각 1, -1을 리턴합니다.
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### Permutation and Random Sampling

In [88]:
df = DataFrame(np.arange(5 * 4).reshape(5, 4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [89]:
# 샘플링할 인덱스를 만들어서 샘플링을 합니다.
sampler = np.random.permutation(5)
print sampler, sampler[:3]
df.take(sampler)[:3]

[1 0 2 3 4] [1 0 2]


,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11


In [90]:
## randint를 이용해서 샘플링을 해보겠습니다.
bag = np.array([0, .1, .2, .3, .4])
sampler = np.random.randint(0, len(bag), size=10)
print sampler
draws = bag.take(sampler)
draws

[2 4 4 4 4 2 2 2 0 3]


array([ 0.2,  0.4,  0.4,  0.4,  0.4,  0.2,  0.2,  0.2,  0. ,  0.3])

### Computing Indicator/Dummy Variables

In [91]:
df = DataFrame({'key': ['비', '비', '맑음', '눈', '맑음', '비'], 'data1': range(6)})
df

,data1,key
0,0,비
1,1,비
2,2,맑음
3,3,눈
4,4,맑음
5,5,비


In [92]:
# 컬럼명이 'key'인 컬럼의 값의 종류별로 컬럼명으로 하여, 그 값이 0아니면 1인 새로운 컬럼을 추가합니다.
# 아래는 '맑음', '비', '눈' 컬럼이 생성되고 그 중 하나의 컬럼 값만이 1이 됩니다.
# 1이되는 컬럼은 기존의 'key' 컬럼에서의 값에 해당하는 컬럼입니다.
pd.get_dummies(df['key'])

,눈,맑음,비
0,0,0,1
1,0,0,1
2,0,1,0
3,1,0,0
4,0,1,0
5,0,0,1


In [93]:
# join을 통해 확인해 볼 수 있습니다.
dummies = pd.get_dummies(df['key'])
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,눈,맑음,비
0,0,0,0,1
1,1,0,0,1
2,2,0,1,0
3,3,1,0,0
4,4,0,1,0
5,5,0,0,1


In [94]:
## 아래와 같이 영화 장르 정보가 각각 문자열로 구분자 '|'를 이용하여 들어가 있습니다.

movies_str = """Toy Story (1995),Animation|Children's|Comedy
Jumanji (1995),Adventure|Children's|Fantasy
Grumpier Old Men (1995),Comedy|Romance
Waiting to Exhale (1995),Comedy|Drama
Father of the Bride Part II (1995),Comedy
Heat (1995),Action|Crime|Thriller
Sabrina (1995),Comedy|Romance
Tom and Huck (1995),Adventure|Children's
Sudden Death (1995),Action
GoldenEye (1995),Action|Adventure|Thriller"""

movies_id = range(1, 11)
movies_title = [line.split(',')[0] for line in movies_str.split('\n')]
movies_genres = [line.split(',')[1] for line in movies_str.split('\n')]

movies = DataFrame({'Movie_id': movies_id, 'Title': movies_title, 'genres': movies_genres})
movies

,Movie_id,Title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [95]:
## 이럴 경우 get_dummies 함수를 사용하기는 곤란하고 아래와 같이 좀 복잡합니다.

genre_iter = (set(x.split('|')) for x in movies.genres)
genres = sorted(set.union(*genre_iter))
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)
for i, gen in enumerate(movies.genres):
    dummies.ix[i, gen.split('|')] = 1
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic

,Movie_id,Title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Drama,Genre_Fantasy,Genre_Romance,Genre_Thriller
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,1,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,0,1,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,1,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,0,0,0
5,6,Heat (1995),Action|Crime|Thriller,1,0,0,0,0,1,0,0,0,1
6,7,Sabrina (1995),Comedy|Romance,0,0,0,0,1,0,0,0,1,0
7,8,Tom and Huck (1995),Adventure|Children's,0,1,0,1,0,0,0,0,0,0
8,9,Sudden Death (1995),Action,1,0,0,0,0,0,0,0,0,0
9,10,GoldenEye (1995),Action|Adventure|Thriller,1,1,0,0,0,0,0,0,0,1


In [96]:
### get_dummies와 cut을 이용하면 아래와 같이 discretization을 할 수 있습니다.
values = np.random.rand(10)
print values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

[ 0.53614715  0.00426885  0.97467944  0.44289236  0.65158907  0.61562033
  0.53089883  0.13941997  0.48344916  0.11734259]


,"(0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1]"
0,0,0,1,0,0
1,1,0,0,0,0
2,0,0,0,0,1
3,0,0,1,0,0
4,0,0,0,1,0
5,0,0,0,1,0
6,0,0,1,0,0
7,1,0,0,0,0
8,0,0,1,0,0
9,1,0,0,0,0


## String Manipulation
### String Object Methods

In [97]:
## split
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [98]:
## strip
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [99]:
## join
' ; '.join(pieces)

'a ; b ; guido'

In [100]:
## in (operator)
'guido' in val

True

In [101]:
## index and find
print val.index(',')    # 찾지 못하면 ValueError 예외를 발생시킵니다. 반드시 있다고 확신하는 경우에만 사용합니다.
print val.find(';')    # 찾지 못하면 -1을 리턴합니다.

1
-1


In [102]:
## count
val.count(',')

2

In [103]:
## replace
val.replace(',', ';')

'a;b; guido'

### Regular expressions

In [104]:
import re
text = 'foo    bar\t baz  \tqux'

In [105]:
## split
## 아래 두 split의 결과는 같습니다.
re.split('\s+', text)    # 정규식을 실시간으로 컴파일해서 사용
regex = re.compile('\s+')    # 정규식을 미리 컴파일 해 둠
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [106]:
## findall
regex.findall(text)

['    ', '\t ', '  \t']

In [107]:
## 텍스트로부터 이메일 패턴(정규식)을 이용해 추출합니다.

text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [108]:
## search
## search의 결과는 맨 처음 일치한 매칭 객체
m = regex.search(text)
print m
text[m.start():m.end()]

'dave@google.com'

In [109]:
## match
## 문자열의 맨 처음부터 일치하지 않으면 None을 리턴
print regex.match(text)

None


In [110]:
## sub
print regex.sub('REDACTED', text)

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [111]:
### 괄호로 싸주면 부분 일치한 영역 별로 그룹을 지어줍니다.

pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [112]:
### findall
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [113]:
### sub
### 순서대로 생성된 placeholder를 통해 값을 참조할 수도 있습니다.
print regex.sub(r'{user: \1}, {domain: \2}, {suffix: \3}', text)

Dave {user: dave}, {domain: google}, {suffix: com}
Steve {user: steve}, {domain: gmail}, {suffix: com}
Rob {user: rob}, {domain: gmail}, {suffix: com}
Ryan {user: ryan}, {domain: yahoo}, {suffix: com}



In [114]:
### placeholder의 이름을 지정해 주면 그룹을 튜플이 아닌 사전으로 받을 수도 있습니다.

regex = re.compile(r"""
(?P<user>[A-Z0-9._%+-]+)
@
(?P<domain>[A-Z0-9.-]+)
\.
(?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)
m = regex.match('wesm@bright.net')
m.groupdict()

{'domain': 'bright', 'suffix': 'net', 'user': 'wesm'}

### Vectorized string functions in pandas

In [115]:
## 문자열 컬럼에 대해 수행하는 메소드들도 있습니다.
data = Series({'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan})
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [116]:
## str.contains
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [117]:
## str.findall
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

In [118]:
## str.match
## pandas 차기 버전에서는 group 객체가 아닌 매치 여부를 boolean 인덱스로 준다고 합니다.
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

/Users/Jamie/usr/virenv.mac/lib/python2.7/site-packages/pandas/core/strings.py:380: UserWarning: In future versions of pandas, match will change to always return a bool indexer.
  " always return a bool indexer.", UserWarning)


Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

In [119]:
## 매칭 객체의 .str 멤버는 튜플이므로, 아래 두 방식으로 접근이 가능합니다.
matches.str.get(1)
matches.str[1]

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

In [120]:
## Series 객체의 값이 문자열일 경우 slicing도 가능합니다.
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object